# For company humAIn an NLP model being a NNMF topic model

# Non-Negative Matric Factorization

Let's repeat thet opic modeling task from the previous lecture, but this time, we will use NMF instead of LDA.

## Step 1: loading data


In [1]:
# Importing modules
import pandas as pd
import os

import json 

In [2]:
# os.chdir('..')

cwd = os.getcwd()
print(cwd)

/home/christopheschellinck/Documents/Projects/project_NLP_humain/ipynb_files


In [3]:
with open("../news_data.json", "r") as f:
    papers10 = json.load(f)

In [4]:
from pandas.io.json import json_normalize

In [5]:
papers = pd.json_normalize(papers10["data"])

In [6]:
papers.head()

,id,title,summary,authors,tags,text,url,source,created_at,updated_at,author,date
0,10813,"ZingBox aims for ‘Internet of Trusted Things’,...",Cybersecurity provider ZingBox has announced t...,None,device\niot\nguardian\napproach\ndevices\nindu...,Cybersecurity provider ZingBox has announced t...,https://artificialintelligence-news.com/2017/0...,AInews,2020-02-05T17:08:34.343Z,2020-02-05T17:08:34.343Z,James Bourne,2017-04-25
1,10814,AI may help create more sustainable data centres,Enterprise data centre provider Aegis Data arg...,None,data\ncentre\nnatural\nnew\ntechnology\nindust...,Enterprise data centre provider Aegis Data arg...,https://artificialintelligence-news.com/2017/0...,AInews,2020-02-05T17:08:34.355Z,2020-02-05T17:08:34.355Z,James Bourne,2017-04-25
2,10815,Why a potential trillion dollar B2B bots indus...,"From Domino’s Pizza, to Uber, to Bank of Ameri...",None,next\nbig\ngupshup\none\nbusiness\ntech\nimpac...,"From Domino’s Pizza, to Uber, to Bank of Ameri...",https://artificialintelligence-news.com/2017/0...,AInews,2020-02-05T17:08:34.365Z,2020-02-05T17:08:34.365Z,James Bourne,2017-04-25
3,10816,Why companies investing in AI today should exp...,Organisations investing in artificial intellig...,None,ai\norganisations\nindustry\nemployees\nexpo\n...,Organisations investing in artificial intellig...,https://artificialintelligence-news.com/2017/0...,AInews,2020-02-05T17:08:34.375Z,2020-02-05T17:08:34.375Z,James Bourne,2017-04-25
4,10817,Tencent gears up for greater GPU acceleration ...,Tencent’s cloud computing services will be bee...,None,gpu\naccelerators\ngpus\ncloud\nservices\ntesl...,Tencent’s cloud computing services will be bee...,https://artificialintelligence-news.com/2017/0...,AInews,2020-02-05T17:08:34.385Z,2020-02-05T17:08:34.385Z,James Bourne,2017-04-26


In [7]:
papers.columns

Index(['id', 'title', 'summary', 'authors', 'tags', 'text', 'url', 'source',
       'created_at', 'updated_at', 'author', 'date'],
      dtype='object')

In [8]:
papers.shape

(1626, 12)

In [9]:
papers.sample(1).text

490    Woody Bledsoe was sitting in a wheelchair in h...
Name: text, dtype: object

Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Step 2: Data cleaning

In [10]:
papers1 = papers

In [11]:
# Remove the columns
papers2 = papers1.drop(columns=['authors', 'url', 'source', 'created_at', 'updated_at', 'author', 'date'], axis=1, inplace = False)

# Print out the first rows of papers
papers2.head()


,id,title,summary,tags,text
0,10813,"ZingBox aims for ‘Internet of Trusted Things’,...",Cybersecurity provider ZingBox has announced t...,device\niot\nguardian\napproach\ndevices\nindu...,Cybersecurity provider ZingBox has announced t...
1,10814,AI may help create more sustainable data centres,Enterprise data centre provider Aegis Data arg...,data\ncentre\nnatural\nnew\ntechnology\nindust...,Enterprise data centre provider Aegis Data arg...
2,10815,Why a potential trillion dollar B2B bots indus...,"From Domino’s Pizza, to Uber, to Bank of Ameri...",next\nbig\ngupshup\none\nbusiness\ntech\nimpac...,"From Domino’s Pizza, to Uber, to Bank of Ameri..."
3,10816,Why companies investing in AI today should exp...,Organisations investing in artificial intellig...,ai\norganisations\nindustry\nemployees\nexpo\n...,Organisations investing in artificial intellig...
4,10817,Tencent gears up for greater GPU acceleration ...,Tencent’s cloud computing services will be bee...,gpu\naccelerators\ngpus\ncloud\nservices\ntesl...,Tencent’s cloud computing services will be bee...


In [12]:
papers2.shape

(1626, 5)

### Applying regex

In [13]:
# Load the regular expression library
import re

# Remove punctuation
papers2['text_preprocessed'] = \
papers2['text'].map(lambda x: re.sub('\s+', ' ', x))

papers2['text_preprocessed'] = \
papers2['text_preprocessed'].map(lambda x: re.sub('[\n]', ' ', x))

papers2['text_preprocessed'] = \
papers2['text_preprocessed'].map(lambda x: re.sub('[\']', '', x))

papers2['text_preprocessed'] = \
papers2['text_preprocessed'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers2['text_preprocessed'] = \
papers2['text_preprocessed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers2['text_preprocessed'].head()

0    cybersecurity provider zingbox has announced t...
1    enterprise data centre provider aegis data arg...
2    from domino’s pizza to uber to bank of america...
3    organisations investing in artificial intellig...
4    tencent’s cloud computing services will be bee...
Name: text_preprocessed, dtype: object

### Applying lemmatization

In [14]:
# Perform standard imports:
import spacy


### spaCy preparation for lemmatization

In [15]:
nlp_en_core_web_sm = spacy.load('en_core_web_sm')

In [16]:
def lemmatizing_article(line):
    
    string = ''
    list1 = []
    doc = nlp_en_core_web_sm(line)
    for token in doc:
        #string = ''.join(token.lemma_)
        list1.append(token.lemma_)
    
    return list1

In [17]:
papers2['text_lemmatized'] = \
papers2['text_preprocessed'].apply(lambda x: lemmatizing_article(x))

In [18]:
print(papers2.head(20))

       id                                              title  \
0   10813  ZingBox aims for ‘Internet of Trusted Things’,...   
1   10814   AI may help create more sustainable data centres   
2   10815  Why a potential trillion dollar B2B bots indus...   
3   10816  Why companies investing in AI today should exp...   
4   10817  Tencent gears up for greater GPU acceleration ...   
5   10818  Bonsai launches Early Access Program to help e...   
6   10819  AI falls on the final furlong in predicting Ke...   
7   10820  Most Britons want AI to support at least part ...   
8   10821  Medicine, law and IT may be affected by the ri...   
9   10822  Cisco acquires AI firm MindMeld to create more...   
10  10823  University of Cambridge bolsters AI research e...   
11  10824  Cray launches two new CS-Storm accelerated clu...   
12  10825  UNICEF joins Apple, Google, Facebook et al in ...   
13  10826  New intelligent street light software aims to ...   
14  10827  Cylance launches first claime

In [19]:
# Function to convert each line of a dataset column from list to string
def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

In [20]:
papers2['text_lemmatized_string'] = \
papers2['text_lemmatized'].apply(lambda x: listToString(x))

In [21]:
print(papers2.head(20))

       id                                              title  \
0   10813  ZingBox aims for ‘Internet of Trusted Things’,...   
1   10814   AI may help create more sustainable data centres   
2   10815  Why a potential trillion dollar B2B bots indus...   
3   10816  Why companies investing in AI today should exp...   
4   10817  Tencent gears up for greater GPU acceleration ...   
5   10818  Bonsai launches Early Access Program to help e...   
6   10819  AI falls on the final furlong in predicting Ke...   
7   10820  Most Britons want AI to support at least part ...   
8   10821  Medicine, law and IT may be affected by the ri...   
9   10822  Cisco acquires AI firm MindMeld to create more...   
10  10823  University of Cambridge bolsters AI research e...   
11  10824  Cray launches two new CS-Storm accelerated clu...   
12  10825  UNICEF joins Apple, Google, Facebook et al in ...   
13  10826  New intelligent street light software aims to ...   
14  10827  Cylance launches first claime

### spaCy preparation for removing stopwords (automatic removed by spaCy as well as the manual selection of stopwords)

In [22]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

In [23]:
# STOP WORD set in order to add manually stopwords to the stop word list
STOP_WORDS |= {"ai", "artificial", "intelligence"}

In [24]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

In [25]:
def removing_stopwords(line):
    #  "nlp" Object is used to create documents with linguistic annotations.
    my_doc = nlp(line)

    # Create list of word tokens
    token_list = []
    
    for token in my_doc:
        token_list.append(token.text)


    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 

    for word in token_list: 

        lexeme = nlp.vocab[word]
   
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
            
            
    return filtered_sentence
    
    
    

In [26]:
papers2['text_cleaned'] = \
papers2['text_lemmatized_string'].apply(lambda x: removing_stopwords(x))

In [27]:
print(papers2['text_cleaned'].head(10))

0    [cybersecurity, provider, zingbox, announce, l...
1    [enterprise, data, centre, provider, aegis, da...
2    [domino, pizza, uber, bank, america, bots, hot...
3    [organisation, invest, (, ), anticipate, 39, %...
4    [tencent, cloud, computing, service, beef, gpu...
5    [-PRON-, -, base, bonsai, set, engage, enterpr...
6    [kentucky, derby, race, triple, crown, horse, ...
7    [new, survey, commission, uc, expo, event, rev...
8    [gartner, tentative, guideline, 2022, smart, m...
9    [cisco, announce, -PRON-, intent, acquire, min...
Name: text_cleaned, dtype: object


In [28]:
papers2['text_cleaned_string'] = \
papers2['text_cleaned'].apply(lambda x: listToString(x))

In [29]:
print(papers2['text_cleaned_string'].head(20))

0     cybersecurity provider zingbox announce launch...
1     enterprise data centre provider aegis data arg...
2     domino pizza uber bank america bots hot proper...
3     organisation invest ( ) anticipate 39 % revenu...
4     tencent cloud computing service beef gpu accel...
5     -PRON- - base bonsai set engage enterprise ind...
6     kentucky derby race triple crown horse racing ...
7     new survey commission uc expo event reveal 85 ...
8     gartner tentative guideline 2022 smart machine...
9     cisco announce -PRON- intent acquire mindmeld ...
10    leverhulme centre future ( cfi ) join - - prof...
11    cray launch new cs - storm accelerate cluster ...
12    unicef announce -PRON- join partnership ( ) da...
13    tcs digital software & solutions group tata co...
14    australia - base cylance announce general avai...
15    peopleai receive $ 7 million series funding le...
16    ( ) deep learning help analyse image patient o...
17    san francisco - base crowdsource firm crow

## Step 3: Splitting the articles with a training part and a test part 
### Is to be doen now because after tfidf application not possible anymore to add the colum "topic"

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
articles_train, articles_test = train_test_split(papers2, test_size = 0.25)
    

## Step 4: Preprocessing with TfidfVectorizer and fit_transform on the training data

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [33]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')

## Step 4.b:  Applying fit_transform on the training data by TfidfVectorizer 

In [34]:
dtm_train = tfidf.fit_transform(articles_train['text_cleaned_string'])


In [35]:
dtm_train

<1219x12341 sparse matrix of type '<class 'numpy.float64'>'
	with 280265 stored elements in Compressed Sparse Row format>

In [36]:
dtm_train.shape

(1219, 12341)

<h1> <p style="color:purple">Step 5: NMF making the model with the training part of the data (THIS IS THE MODEL) </h1></p>

## Step 5.a: Making the model

In [37]:
from sklearn.decomposition import NMF

In [38]:
nmf_model = NMF(n_components=20,random_state=42)

In [39]:
### fit based on the train data

In [40]:
# This can take awhile, we're dealing with a large amount of documents!
nmf_model.fit(dtm_train)

/home/christopheschellinck/anaconda3/envs/virtualenvchris/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


NMF(n_components=20, random_state=42)

## Step 5.b: Saving the model

In [41]:
import pickle

In [42]:
# Save to file in the current working directory
pkl_filename = "pickle_model_NLP.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(nmf_model, file)

## Step 5.c: Displaying Topics

In [43]:
len(tfidf.get_feature_names())

12341

In [44]:
import random

In [45]:
for i in range(10):
    random_word_id = random.randint(0, len(tfidf.get_feature_names()))
    print(tfidf.get_feature_names()[random_word_id])

arxiv
1982
changer
observe
geological
perfect
aftermath
fatality
gestate
rss


In [46]:
for i in range(10):
    random_word_id = random.randint(0, len(tfidf.get_feature_names()))
    print(tfidf.get_feature_names()[random_word_id])

crossing
1983
upside
dropout
rest
synchrony
rife
underline
extenuate
constantly


In [47]:
len(nmf_model.components_)

20

In [48]:
nmf_model.components_

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.00113023],
       [0.        , 0.0016439 , 0.        , ..., 0.        , 0.        ,
        0.00248592],
       [0.01289503, 0.        , 0.        , ..., 0.        , 0.0009958 ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.0009748 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00067661,
        0.        ]])

In [49]:
len(nmf_model.components_[0])

12341

In [50]:
single_topic = nmf_model.components_[0]

In [51]:
# Returns the indices that would sort this array.
single_topic.argsort()

array([   0, 6930, 6929, ..., 9313, 5501, 7188])

In [52]:
# Word least representative of this topic
single_topic[0]

0.0

In [53]:
# Word most representative of this topic
single_topic[4197]

0.0

In [54]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

array([ 6381, 11726, 11283,  6686,  2987,   694,  7683,  9313,  5501,
        7188])

In [55]:
top_word_indices = single_topic.argsort()[-10:]

In [56]:
for index in top_word_indices:
    print(tfidf.get_feature_names()[index])

learn
use
train
machine
datum
algorithm
object
researcher
image
model


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [57]:
print(nmf_model.components_)

[[0.         0.         0.         ... 0.         0.         0.00113023]
 [0.         0.0016439  0.         ... 0.         0.         0.00248592]
 [0.01289503 0.         0.         ... 0.         0.0009958  0.        ]
 ...
 [0.         0.         0.         ... 0.0009748  0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.00067661 0.        ]]


In [58]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print([topic[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['data', 'visual', 'paper', 'computer', 'video', 'learn', 'use', 'train', 'machine', 'datum', 'algorithm', 'object', 'researcher', 'image', 'model']
[0.3131892785469894, 0.33459312842558175, 0.3628554314103979, 0.3849990459943994, 0.4031714569958412, 0.41955387766463825, 0.4334422843634665, 0.4404601884476949, 0.44805091233281036, 0.5750103140681402, 0.5833539462127781, 0.6176098115151633, 0.6316017911461088, 0.9861454950986844, 1.3873559536117015]


THE TOP 15 WORDS FOR TOPIC #1
['use', 'security', 'cyber', 'attend', 'blockchain', 'cloud', 'upcoming', 'enterprise', 'amsterdam', 'london', 'industry', 'leader', 'iot', 'locate', 'expo']
[0.2559943499440473, 0.25923165742098225, 0.262226437031841, 0.26371340285121414, 0.2656544595037193, 0.26676208080789277, 0.2672390609743835, 0.26982473251342093, 0.26989580611700215, 0.284607214685756, 0.2869235868888936, 0.2889771340962341, 0.2976264921647239, 0.4838945333346996, 1.118062876466104]


THE TOP 15 WORDS FOR T

['dedicated', '980', 'pro', '970', 'flagship', 'assistant', 'feature', 'device', 'expo', 'samsung', 'npu', 'smartphone', 'kirin', 'mate', 'huawei']
[0.10486813477716841, 0.10772063284650484, 0.11083766315013421, 0.11651838560135311, 0.13146280210607086, 0.14214818321289316, 0.14436731122985547, 0.14867037655552282, 0.1506092612944287, 0.2086334525533099, 0.2210616754576454, 0.22911747357723636, 0.25009988816667994, 0.32428558752745956, 1.1473978926753026]


THE TOP 15 WORDS FOR TOPIC #18
['startup', 'tesla', 'researcher', 'profit', 'sutskever', 'world', 'software', 'dota', 'research', 'elon', 'brockman', 'agi', 'altman', 'musk', 'openai']
[0.08945373232009196, 0.09812508184993035, 0.10086115810882679, 0.10536015539560296, 0.10657595935416325, 0.11235384013366642, 0.1126435084202151, 0.12192644817325514, 0.12590659375612084, 0.15721826345189155, 0.18632472382391704, 0.18654749109153576, 0.3216810952034004, 0.42428107158491507, 0.9586525464290213]


THE TOP 15 WORDS FOR TOPIC #19
['talen

In [59]:
dfs = []
for index,topic in enumerate(nmf_model.components_):
#    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    names = [tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]]
    weight = [topic[i] for i in topic.argsort()[-15:]]
    d = {'Names' : names, 'Weight' : weight}
    df = pd.DataFrame(d)
    df = df.sort_values(by='Weight', ascending=False)
    dfs.append(df)


In [60]:
dfs[0].head(5)

,Names,Weight
14,model,1.387356
13,image,0.986145
12,researcher,0.631602
11,object,0.617610
10,algorithm,0.583354


# Attaching Discovered Topic Labels to Original Articles

In [61]:
articles_train.shape

(1219, 10)

In [62]:
articles_test.shape

(407, 10)

### transform based on the train data

In [63]:
topic_results_train = nmf_model.transform(dtm_train)

In [64]:
topic_results_train.shape

(1219, 20)

In [65]:
topic_results_train[0]

array([6.64200502e-03, 0.00000000e+00, 0.00000000e+00, 1.06705308e-01,
       1.32153810e-02, 0.00000000e+00, 8.75049018e-02, 0.00000000e+00,
       0.00000000e+00, 1.25578658e-04, 2.22626876e-01, 0.00000000e+00,
       0.00000000e+00, 1.24267826e-02, 0.00000000e+00, 0.00000000e+00,
       7.81546072e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00])

In [66]:
topic_results_train[0].round(2)

array([0.01, 0.  , 0.  , 0.11, 0.01, 0.  , 0.09, 0.  , 0.  , 0.  , 0.22,
       0.  , 0.  , 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ])

In [67]:
topic_results_train[0].argmax()

10

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [68]:
papers2.head()

,id,title,summary,tags,text,text_preprocessed,text_lemmatized,text_lemmatized_string,text_cleaned,text_cleaned_string
0,10813,"ZingBox aims for ‘Internet of Trusted Things’,...",Cybersecurity provider ZingBox has announced t...,device\niot\nguardian\napproach\ndevices\nindu...,Cybersecurity provider ZingBox has announced t...,cybersecurity provider zingbox has announced t...,"[cybersecurity, provider, zingbox, have, annou...",cybersecurity provider zingbox have announce t...,"[cybersecurity, provider, zingbox, announce, l...",cybersecurity provider zingbox announce launch...
1,10814,AI may help create more sustainable data centres,Enterprise data centre provider Aegis Data arg...,data\ncentre\nnatural\nnew\ntechnology\nindust...,Enterprise data centre provider Aegis Data arg...,enterprise data centre provider aegis data arg...,"[enterprise, data, centre, provider, aegis, da...",enterprise data centre provider aegis data arg...,"[enterprise, data, centre, provider, aegis, da...",enterprise data centre provider aegis data arg...
2,10815,Why a potential trillion dollar B2B bots indus...,"From Domino’s Pizza, to Uber, to Bank of Ameri...",next\nbig\ngupshup\none\nbusiness\ntech\nimpac...,"From Domino’s Pizza, to Uber, to Bank of Ameri...",from domino’s pizza to uber to bank of america...,"[from, domino, ’s, pizza, to, uber, to, bank, ...",from domino ’s pizza to uber to bank of americ...,"[domino, pizza, uber, bank, america, bots, hot...",domino pizza uber bank america bots hot proper...
3,10816,Why companies investing in AI today should exp...,Organisations investing in artificial intellig...,ai\norganisations\nindustry\nemployees\nexpo\n...,Organisations investing in artificial intellig...,organisations investing in artificial intellig...,"[organisation, invest, in, artificial, intelli...",organisation invest in artificial intelligence...,"[organisation, invest, (, ), anticipate, 39, %...",organisation invest ( ) anticipate 39 % revenu...
4,10817,Tencent gears up for greater GPU acceleration ...,Tencent’s cloud computing services will be bee...,gpu\naccelerators\ngpus\ncloud\nservices\ntesl...,Tencent’s cloud computing services will be bee...,tencent’s cloud computing services will be bee...,"[tencent, ’s, cloud, computing, service, will,...",tencent ’s cloud computing service will be bee...,"[tencent, cloud, computing, service, beef, gpu...",tencent cloud computing service beef gpu accel...


In [69]:
papers2.tail()

,id,title,summary,tags,text,text_preprocessed,text_lemmatized,text_lemmatized_string,text_cleaned,text_cleaned_string
1621,12434,Robotic pets may be bad medicine for melancholy,Sherry Turkle finds human-machine love unsettling,Technology and society\nResearch Laboratory of...,"In the face of techno-doomsday punditry, Sherr...",in the face of techno-doomsday punditry sherry...,"[in, the, face, of, techno, -, doomsday, pundi...",in the face of techno - doomsday punditry sher...,"[face, techno, -, doomsday, punditry, sherry, ...",face techno - doomsday punditry sherry turkle ...
1622,12435,MIT develops Anklebot for stroke patients,Research team foresees robotic gym,arm\ntrial\ntherapy\nclinical\nhogan\nmedical\...,Clinical trials have already shown that an MIT...,clinical trials have already shown that an mit...,"[clinical, trial, have, already, show, that, a...",clinical trial have already show that an mit r...,"[clinical, trial, mit, robotic, arm, help, str...",clinical trial mit robotic arm help stroke pat...
1623,12436,Notes from the Lab,UNDERWATER ROBOTS THAT MAP AND NAVIGATE,Research Laboratory of Electronics\nmit\nImagi...,Imagine driving down an unfamiliar road and tr...,imagine driving down an unfamiliar road and tr...,"[imagine, drive, down, an, unfamiliar, road, a...",imagine drive down an unfamiliar road and try ...,"[imagine, drive, unfamiliar, road, try, find, ...",imagine drive unfamiliar road try find -PRON- ...
1624,12437,Ray and Maria Stata give $25 million to MIT,Gift is largest ever for Institute building pr...,together\nnew\nCampus buildings and architectu...,MIT today announced a $25 million donation by ...,mit today announced a $25 million donation by ...,"[mit, today, announce, a, $, 25, million, dona...",mit today announce a $ 25 million donation by ...,"[mit, today, announce, $, 25, million, donatio...",mit today announce $ 25 million donation ray m...
1625,12438,Reuters Uses AI To Prototype First Ever Automa...,AI is coming for journalism. But rather than s...,journalism\ncontent generation\nreuters\nSynth...,AI is coming for journalism. But rather than s...,ai is coming for journalism but rather than si...,"[ai, be, come, for, journalism, but, rather, t...",ai be come for journalism but rather than simp...,"[come, journalism, simply, use, job, writer, r...",come journalism simply use job writer reuters ...


In [70]:
topic_results_train.argmax(axis=1)

array([10, 15,  1, ..., 19,  4, 13])

In [71]:
articles_train['Topic'] = topic_results_train.argmax(axis=1)

/home/christopheschellinck/anaconda3/envs/virtualenvchris/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [72]:
print(articles_train)

         id                                              title  \
523   11334  Teaching Self-Driving Cars to Watch for Unpred...   
846   11659  Google’s AI Wizard Unveils a New Twist on Neur...   
477   11289  Microsoft details $40 million ‘AI for Health’ ...   
562   11373  The Style Maven Astrophysicists of Silicon Valley   
1544  12357                                           Car talk   
...     ...                                                ...   
1621  12434    Robotic pets may be bad medicine for melancholy   
808   11620  To Advance Artificial Intelligence, Reverse-En...   
272   11084  Apple promotes AI engineer poached from Google...   
1477  12289  IBM and MIT to pursue joint research in artifi...   
372   11185  Accenture: How automation, augmentation and in...   

                                                summary  \
523   You drive differently for Sunday errands than ...   
846   Google's Geoff Hinton helped catalyze the curr...   
477   Microsoft is reminding u

## <p style="color:purple">Step 6: Using the trained model to define the topics on an imported PDF-file </p>

# Step X: topic results with the PDF-file

## Step X.a:  Importing PDF file

In [73]:
# note the capitalization
import PyPDF2

from PyPDF2 import PdfFileReader

import os
# import str

In [74]:
os.chdir('../reports')

cwd = os.getcwd()
print(cwd)

/home/christopheschellinck/Documents/Projects/project_NLP_humain/reports


In [106]:
url_MGI = 'MGI-The-Age-of-Analytics-Full-report.pdf'
url_DS_use_cases ='The Big Book of Data Science Use Cases.pdf'
url_big_Data = 'Using big data to make better pricing decisions.pdf'
url_AI_strat = 'aiadoptionstrategies-march2019pdf.pdf'
url_main = 'main.pdf'

In [115]:
# pdf file object
# you can find find the pdf file with complete code in below


def extract_text_from_pdf(url):
    pdfFileObj = open(url, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    text = ''
    for i in range(0, pdfReader.numPages):
        pageObj = pdfReader.getPage(i)
        text += pageObj.extractText()
    f.close()
    return text



In [162]:
text = extract_text_from_pdf(url_MGI)


In [163]:
print(text)

Organizational 
challenges 
Disruptive business 

models 
Enhanced decision 

making 
755534HIGHLIGHTSIN COLLABORATION WITH
MCKINSEY ANALYTICS
DECEMBER 2016THE AGE OF ANALYTICS:
 COMPETING IN A  DATA-DRIVEN WORLD
Copyright © McKinsey & Company 2016In the 25˜years since its founding, the McKinsey Global Institute (MGI) has sought to develop 
a deeper understanding of the evolving global economy. As the business and economics 
research arm of McKinsey & Company, MGI aims to provide leaders in the commercial, 
public, and social sectors with the facts and insights on which to base management and 

policy decisions. The Lauder Institute at the University of Pennsylvania ranked MGI the 

world™s number-one private-sector think tank in its 2015 Global Think Tank Index. 
MGI research combines the disciplines of economics and management, employing the 
analytical tools of economics with the insights of business leaders. Our ﬁmicro-to-macroﬂ 
methodology examines microeconomic industry trends t

## Step X.b:  Applying transform on the test article string by TfidfVectorizer

In [198]:
dtm_test = tfidf.transform(text_dataframe['content_cleaned'])


In [199]:
dtm_test.shape

(8859, 12341)

## topics with the test data

### transform based on the test data

In [86]:
topic_results_test = nmf_model.transform(dtm_test)

In [87]:
topic_results_test.shape

(1, 20)

In [88]:
topic_results_test[0]

array([0.00307458, 0.00747218, 0.        , 0.00010829, 0.003142  ,
       0.00193241, 0.00330788, 0.00646542, 0.00359976, 0.        ,
       0.0016042 , 0.        , 0.        , 0.00525557, 0.01328972,
       0.00446471, 0.00815731, 0.01645819, 0.        , 0.00283232])

In [89]:
print(topic_results_test[0])

[0.00307458 0.00747218 0.         0.00010829 0.003142   0.00193241
 0.00330788 0.00646542 0.00359976 0.         0.0016042  0.
 0.         0.00525557 0.01328972 0.00446471 0.00815731 0.01645819
 0.         0.00283232]


In [90]:
topic_results_test[0].round(4)

array([0.0031, 0.0075, 0.    , 0.0001, 0.0031, 0.0019, 0.0033, 0.0065,
       0.0036, 0.    , 0.0016, 0.    , 0.    , 0.0053, 0.0133, 0.0045,
       0.0082, 0.0165, 0.    , 0.0028])

### the string of the file is most near topic number 11

In [91]:
topic_results_test[0].argmax()

17

In [92]:
dtm_test.shape

(1, 12341)